# Compute climate index DD ('drydays'). The computation in the back is taken from the python module xclim

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import warnings

warnings.filterwarnings("ignore")

In [29]:
import glob
from datetime import date

import xarray as xr

In [7]:
from pyhomogenize import open_xrdataset

import index_calculator as xcalc

Select one CORDEX `tas` file and open it with xarray.

In [4]:
ifile = "tas_EUR-11_MPI-M-MPI-ESM-LR_historical_r3i1p1_GERICS-REMO2015_v1_day_20010101-20051231.nc"

In [8]:
ifile = "/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/2003/2003025.00.nc"

In [11]:
path = "/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA"

In [31]:
start = date(1997, 1, 1)
end = date(1998, 1, 1)

yearList = [year for year in range(start.year, end.year)]  # + 1)]

pr = []

for year in yearList:
    pr += glob.glob(path + "/" + str(year) + "/*" + str(year) + "*.nc")

pr

['/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1997/1997333.00.nc',
 '/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1997/1997230.12.nc',
 '/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1997/1997338.15.nc',
 '/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1997/1997157.00.nc',
 '/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1997/1997278.03.nc',
 '/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1997/1997318.06.nc',
 '/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1997/1997268.09.nc',
 '/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1997/1997248.15.nc',
 '/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1997/1997066.00.nc',
 '/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1997/1997144.21.nc',
 '/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1997/1997329.15.nc',
 '/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1997/1997240.03.nc',
 '/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1997/1997065.18.nc',
 '/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1997/1997123.21.nc',
 '/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1997/1997060.00.nc',
 '/work/ch

In [32]:
ds = xr.open_mfdataset(pr)

In [34]:
ds

<xarray.Dataset>
Dimensions:        (lon: 3600, lat: 1800, time: 2920)
Coordinates:
  * lon            (lon) float32 -179.9 -179.8 -179.8 ... 179.8 179.9 179.9
  * lat            (lat) float32 89.95 89.85 89.75 ... -89.75 -89.85 -89.95
  * time           (time) datetime64[ns] 1997-01-01 ... 1997-12-31T21:00:00
Data variables:
    precipitation  (time, lat, lon) float32 dask.array<chunksize=(1, 1800, 3600), meta=np.ndarray>
Attributes:
    history:  Created on 2021-12-27 14:18

Now do a preprocessing of the dataset and create an index_calculator.PreProcessing object.
Preprocessor tasks are:
* select a user-given time range (parameter: time_range)
* check time axis for missing, redundant and duplicated time steps
* crop time axis to required left and right border
* write available and requested time range to object

In [35]:
preproc = xcalc.preprocessing(ds)
preproc.preproc

UnboundLocalError: local variable 'coords' referenced before assignment

Now, use the above object to compute the climate index `DD`.

In [7]:
proc = xcalc.processing("DD", preproc_obj=preproc)
proc.proc

<xarray.Dataset>
Dimensions:                     (rlat: 412, rlon: 424, vertices: 4, time: 5,
                                 bounds: 2)
Coordinates:
    height                      float64 2.0
  * rlat                        (rlat) float64 -23.38 -23.27 ... 21.72 21.84
  * rlon                        (rlon) float64 -28.38 -28.27 ... 18.04 18.16
    lat                         (rlat, rlon) float32 21.99 22.03 ... 66.75 66.69
    lon                         (rlat, rlon) float32 349.9 350.0 ... 64.76 64.96
  * time                        (time) object 2001-07-03 00:00:00 ... 2005-07...
Dimensions without coordinates: vertices, bounds
Data variables:
    rotated_latitude_longitude  int32 ...
    lat_vertices                (rlat, rlon, vertices) float32 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>
    lon_vertices                (rlat, rlon, vertices) float32 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>
    TG                          (time, rlat, rlon) float32 dask.array<chunksize=(1, 412, 424), meta=np.ndarray>
    time_bnds                   (time, bounds) object 2001-01-01 12:00:00 ......
Attributes: (12/35)
    institution:                    Helmholtz-Zentrum Geesthacht, Climate Ser...
    institute_id:                   GERICS
    experiment_id:                  historical
    source:                         GERICS-REMO2015
    model_id:                       GERICS-REMO2015
    forcing:                        N/A
    ...                             ...
    parent_experiment:              N/A
    modeling_realm:                 atmos
    realization:                    3
    cmor_version:                   2.9.1
    tracking_id:                    hdl:21.14103/55a844fd-666d-4799-8563-8fbd...
    CF_variables:                   ['tas']

Finally, do some postprocessing. The postprocessor writes some additional attributes to `ds`' global attributes.

In [8]:
postproc = xcalc.postprocessing(
    project="MSWEP",
    proc_obj=proc,
    institution="Helmholtz-Zentrum hereon GmbH, Climate Service Center Germany",
    institution_id="GERICS",
)
postproc.postproc

<xarray.Dataset>
Dimensions:                     (rlat: 412, rlon: 424, vertices: 4, time: 5,
                                 bounds: 2)
Coordinates:
    height                      float64 2.0
  * rlat                        (rlat) float64 -23.38 -23.27 ... 21.72 21.84
  * rlon                        (rlon) float64 -28.38 -28.27 ... 18.04 18.16
    lat                         (rlat, rlon) float32 21.99 22.03 ... 66.75 66.69
    lon                         (rlat, rlon) float32 349.9 350.0 ... 64.76 64.96
  * time                        (time) object 2001-07-03 00:00:00 ... 2005-07...
Dimensions without coordinates: vertices, bounds
Data variables:
    rotated_latitude_longitude  int32 ...
    lat_vertices                (rlat, rlon, vertices) float32 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>
    lon_vertices                (rlat, rlon, vertices) float32 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>
    TG                          (time, rlat, rlon) float32 dask.array<chunksize=(1, 412, 424), meta=np.ndarray>
    time_bnds                   (time, bounds) object 2001-01-01 12:00:00 ......
Attributes: (12/45)
    institution:                    Helmholtz-Zentrum Geesthacht, Climate Ser...
    institute_id:                   GERICS
    experiment_id:                  historical
    source:                         GERICS-REMO2015
    model_id:                       GERICS-REMO2015
    forcing:                        N/A
    ...                             ...
    ci_name:                        TG
    ci_frequency:                   year
    ci_timerange_index:             2001-2005
    ci_timerange_source:            20010101-20051231
    ci_package_name:                xclim_0.37.0
    ci_package_reference:           xcalc_0.3.2

Optionaly, we can store the dataset as a netCDF file on disk. The output file naming depends on the user-given project. 

In [9]:
xcalc.outputwriter(postproc_obj=postproc)

File written: ./cordex/climdex/EUR-11/GERICS/GERICS/MPI-M-MPI-ESM-LR/historical/r3i1p1/GERICS-REMO2015/v1/year/TG/TG_EUR-11_MPI-M-MPI-ESM-LR_historical_r3i1p1_GERICS-REMO2015_v1_day_GERICS_year_2001-2005.nc


For simplyfication, we can do preprocessing, processing, postprocessing and output file storing by calling just one function.

In [10]:
idx_obj = xcalc.index_calculator(
    ds=ds,
    index="DD",
    project="MSWEP",
    institution_id="GERICS",
    institution="Helmholtz-Zentrum hereon GmbH, Climate Service Center Germany",
    write=True,
)

File written: ./cordex/climdex/EUR-11/GERICS/GERICS/MPI-M-MPI-ESM-LR/historical/r3i1p1/GERICS-REMO2015/v1/year/TG/TG_EUR-11_MPI-M-MPI-ESM-LR_historical_r3i1p1_GERICS-REMO2015_v1_day_GERICS_year_2001-2005.nc


In [11]:
idx_obj.postproc

<xarray.Dataset>
Dimensions:                     (rlat: 412, rlon: 424, vertices: 4, time: 5,
                                 bounds: 2)
Coordinates:
    height                      float64 2.0
  * rlat                        (rlat) float64 -23.38 -23.27 ... 21.72 21.84
  * rlon                        (rlon) float64 -28.38 -28.27 ... 18.04 18.16
    lat                         (rlat, rlon) float32 21.99 22.03 ... 66.75 66.69
    lon                         (rlat, rlon) float32 349.9 350.0 ... 64.76 64.96
  * time                        (time) object 2001-07-03 00:00:00 ... 2005-07...
Dimensions without coordinates: vertices, bounds
Data variables:
    rotated_latitude_longitude  int32 ...
    lat_vertices                (rlat, rlon, vertices) float32 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>
    lon_vertices                (rlat, rlon, vertices) float32 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>
    TG                          (time, rlat, rlon) float32 dask.array<chunksize=(1, 412, 424), meta=np.ndarray>
    time_bnds                   (time, bounds) object 2001-01-01 12:00:00 ......
Attributes: (12/45)
    institution:                    Helmholtz-Zentrum Geesthacht, Climate Ser...
    institute_id:                   GERICS
    experiment_id:                  historical
    source:                         GERICS-REMO2015
    model_id:                       GERICS-REMO2015
    forcing:                        N/A
    ...                             ...
    ci_name:                        TG
    ci_frequency:                   year
    ci_timerange_index:             2001-2005
    ci_timerange_source:            20010101-20051231
    ci_package_name:                xclim_0.37.0
    ci_package_reference:           xcalc_0.3.2